## GPT (Generative Pre-Trained Transformer) 

<img src="images/gpt_stable_dif.png" width="20%" height="20%">

GPT, ilk olarak OpenAI tarafından [Improving Language Understanding by Generative Pre-Training by Alec Radford et al. in 2018](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf) makalesinde tanıtılan Transformer tabanlı bir dil modelidir.

Bu notebook'ta, GPT Modelini PyTorch kullanarak implement edeceğiz

Not: Model mimarisine ek olarak özellikle eğitim yöntemleri ile dikkat çekmektedir. Bu nedenle modelin nasıl eğitildiğinin ayrıca incelemesinde fayda var ancak bu notebookta biz sadece modelin mimarisine odaklanacağız. İlerleyen zamanlarda mümkün olduğu kadarıyla eğitim yöntemlerini de incelemeye çalışacağız.

## Kütüphaneleri içeri aktarma

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

## Hiperparametreleri tanımlama

In [2]:
# Tüm hiperparametreler makaledeki değerlere göre ayarlanmıştır
embed_dim = 768
head_nums = 12
layer_num = 12
batch_size = 64
block_size = 512
forward_expansion = 4
vocab_size = 40000

attn_dropout = 0.1
dropout_rate = 0.1

## GPT Modelinin bloklarını oluşturuyoruz.

### Multi Head Attention

Bu blokta multi head attention bloğunu implement. Bu blok transformer modelinin en önemli parçasıdır.

Bu bloğun implementasyonunda "*scaled dot product attention*" işlemini sıfırdan implement etmeyeceğim çünkü PyTorch'un kendi implementasyonu [FlashAttention](https://arxiv.org/abs/2205.14135) implementasyonunu kullanmaktadır (PyTorch >= 2.0) ve bu implementasyon bizim kendi yapacağımız klasik implementasyondan çok daha verimli çalışacaktır.

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, attn_drop_rate, proj_drop_rate):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"

        self.qkv = nn.Linear(embed_dim, embed_dim * 3, bias=False)
        self.attn_drop_rate = attn_drop_rate
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.proj_drop = nn.Dropout(proj_drop_rate)

    def forward(self, x):
        B,T,C = x.shape # x: (batch_size, block_size, embedding_dim)
        qkv = self.qkv(x) # x: (batch_size, block_size, embedding_dim*3)
        qkv = qkv.reshape(B,T,3,self.num_heads,self.head_dim).permute(2,0,3,1,4) # x: (3, batch_size, num_heads, block_size, head_dim)
        query,key,value = qkv[0],qkv[1],qkv[2] # query, key, value: (batch_size, num_heads, block_size, head_dim)
        attn = F.scaled_dot_product_attention(query,key,value, dropout_p=self.attn_drop_rate, is_causal=True) # attn: (batch_size, num_heads, block_size, head_dim)
        attn = attn.transpose(1, 2).contiguous().view(B, T, C) # attn: (batch_size, block_size, embedding_dim)
        out = self.proj(attn) # out: (batch_size, block_size, embedding_dim)
        out = self.proj_drop(out)
        return out

### Feed Forward Network

Bu block basitçe 2 gizli katmandan oluşan bir ileri beslemeli ağdır. İleri beslemeli ağ için makalede 3072 boyutlu iç durumlar (**inner_states**) kullanılmıştır. **inner_states** terimi yerine bu implementasyonda daha kolay anlaşılması açısından  "**forward expansion**" terimini kullanacağım ( *embedding_dim* ${*}$ *forward_expansion* = *inner_states* ). Ayrıca orjinal Transformer'dan farklı olarak burada ReLU yerine makalede belirtildiği üzere GELU (Gaussian Error Linear Unit) aktivasyon fonksiyonunu kullanacağız.

In [4]:
class FFN(nn.Module):
    def __init__(self, embedding_dim, forward_expansion, drop_rate):
        super().__init__()
        self.linear_1 = nn.Linear(embedding_dim, embedding_dim*forward_expansion)
        self.gelu = nn.GELU()
        self.linear_2 = nn.Linear(embedding_dim*forward_expansion, embedding_dim)
        self.dropout = nn.Dropout(drop_rate)
    
    def forward(self, x):
        x = self.linear_1(x) # (batch_size, seq_len, embedding_dim*forward_expansion)
        x = self.gelu(x) # (batch_size, seq_len, embedding_dim*forward_expansion)
        x = self.linear_2(x) # (batch_size, seq_len, embedding_dim)
        out = self.dropout(x)
        return out


### GPT Block

In [5]:
class GPTBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, forward_expansion, attn_drop_rate, drop_rate):
        super().__init__()
        self.MHA = MultiHeadAttention(embed_dim=embed_dim, num_heads=num_heads,
                                       attn_drop_rate=attn_drop_rate, proj_drop_rate=drop_rate)
        
        self.FFN = FFN(embedding_dim=embed_dim, forward_expansion=forward_expansion, drop_rate=drop_rate)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(embed_dim)
    
    def forward(self, x):
        # x: (batch_size, seq_len, embed_dim)
        mha_out = self.MHA(x) # (batch_size, seq_len, embed_dim)
        x = self.layer_norm1(mha_out + x)
        ffn_out = self.FFN(x) # (batch_size, seq_len, embed_dim)
        out = self.layer_norm2(ffn_out + x)
        return out


### GPT 

Son olarak bu blokta her şeyi birleştirip GPT modelini tamamlayacağız. GPT modeli geçmiş bloklarda gördüğümüz çeşitli değişiklikler haricinde ek olarak bi de orijinal Transformer modelinden farklı bir positional embedding kullanması ile ayrılır. Orijinal Transformer modeli sinusoidal positional embedding kullanırken biz bu blokta makalede anlatıldığı gibi öğrenilebilir/öğrenilmiş positional embedding kullanacağız.

In [10]:
class GPT(nn.Module):
    def __init__(self, vocab_size ,block_size, embed_dim, num_heads, num_layers, forward_expansion, attn_drop_rate=0., drop_rate=0.):
        super().__init__()

        self.token_embed = nn.Embedding(vocab_size, embed_dim) #  Defining the token embedding layer
        self.pos_embed = nn.Embedding(block_size, embed_dim) # Defining the positional embedding layer

        self.blocks = nn.ModuleList([GPTBlock(embed_dim, num_heads, forward_expansion, attn_drop_rate, drop_rate) for _ in range(num_layers)]) # Defining the GPT blocks (12 layers in the original paper)

        self.linear_head = nn.Linear(embed_dim, vocab_size) # Defining the linear layer to the predictions

        self.dropout = nn.Dropout(drop_rate)

        self.apply(self._init_weights) # Weights initialization

    # We initialize the weights of the model with a normal distribution, with mean 0 and standard deviation 0.02 as in the original paper.
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x):
        B,T = x.shape
        token_embeddings = self.token_embed(x) # shape (B,T,C)
        pos = torch.arange(0, T, dtype=torch.long, device=device) # shape (T)
        pos_embeddings = self.pos_embed(pos) # shape (T,C)
        x = self.dropout(token_embeddings+pos_embeddings) # shape (B,T,C)
        for block in self.blocks:
            x = block (x) # shape (B,T,C)
        out = self.linear_head(x) # shape (B,T,V) where V is the vocab size
        return out

### Son 

Eğer herhangi bir sorunuz olursa bana ulaşabilirsiniz.

- Email: [i_konak@hotmail.com](mailto:i_konak@hotmail.com)
- Linkedin: [Ismail Konak](https://www.linkedin.com/in/ismail-konak/)